In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=f5459bc3573a93dab77a050169924dadbfd47e6c28f2a120f9d63ecae1620157
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RS').getOrCreate()

In [ ]:
df = spark.read.csv("ratings.csv",header=True,inferSchema=True)
df1= spark.read.csv("movies.csv",header=True, inferSchema=True)

In [ ]:
df=df.join(df1, (df["movieId"] == df1["movieId"]))
df=df.drop("timestamp","genres")


In [ ]:
df.count()

100836

In [ ]:
df.columns

['userId', 'movieId', 'rating', 'movieId', 'title']

In [ ]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)



In [ ]:
from pyspark.sql.functions import rand
df.orderBy(rand()).show(10,False)

+------+-------+------+-------+--------------------------------------------------+
|userId|movieId|rating|movieId|title                                             |
+------+-------+------+-------+--------------------------------------------------+
|588   |380    |3.0   |380    |True Lies (1994)                                  |
|561   |8371   |2.0   |8371   |Chronicles of Riddick, The (2004)                 |
|480   |1080   |4.5   |1080   |Monty Python's Life of Brian (1979)               |
|132   |31658  |4.0   |31658  |Howl's Moving Castle (Hauru no ugoku shiro) (2004)|
|94    |153    |3.0   |153    |Batman Forever (1995)                             |
|307   |6593   |2.0   |6593   |Freaky Friday (2003)                              |
|562   |6731   |3.5   |6731   |Day of the Dead (1985)                            |
|51    |804    |5.0   |804    |She's the One (1996)                              |
|483   |2572   |2.5   |2572   |10 Things I Hate About You (1999)                 |
|428

In [ ]:
# Hangi user kac adet icerigi puanlamis?
df.groupBy('userId').count().orderBy('count',ascending=False).show(15,False)

+------+-----+
|userId|count|
+------+-----+
|414   |2698 |
|599   |2478 |
|474   |2108 |
|448   |1864 |
|274   |1346 |
|610   |1302 |
|68    |1260 |
|380   |1218 |
|606   |1115 |
|288   |1055 |
|249   |1046 |
|387   |1027 |
|182   |977  |
|307   |975  |
|603   |943  |
+------+-----+
only showing top 15 rows



In [ ]:
df.groupBy('userId').count().orderBy('count',ascending=True).show(15,False)

+------+-----+
|userId|count|
+------+-----+
|442   |20   |
|53    |20   |
|406   |20   |
|257   |20   |
|431   |20   |
|189   |20   |
|207   |20   |
|278   |20   |
|194   |20   |
|569   |20   |
|320   |20   |
|576   |20   |
|595   |20   |
|147   |20   |
|547   |21   |
+------+-----+
only showing top 15 rows



In [ ]:
# En cok oylanan Filmler hangileri?
df.groupBy('title').count().orderBy('count',ascending=False).show(10,False)

+-----------------------------------------+-----+
|title                                    |count|
+-----------------------------------------+-----+
|Forrest Gump (1994)                      |329  |
|Shawshank Redemption, The (1994)         |317  |
|Pulp Fiction (1994)                      |307  |
|Silence of the Lambs, The (1991)         |279  |
|Matrix, The (1999)                       |278  |
|Star Wars: Episode IV - A New Hope (1977)|251  |
|Jurassic Park (1993)                     |238  |
|Braveheart (1995)                        |237  |
|Terminator 2: Judgment Day (1991)        |224  |
|Schindler's List (1993)                  |220  |
+-----------------------------------------+-----+
only showing top 10 rows



In [ ]:
df.orderBy(rand()).show(10,False)

+------+-------+------+-------+------------------------------------------------------+
|userId|movieId|rating|movieId|title                                                 |
+------+-------+------+-------+------------------------------------------------------+
|307   |8641   |4.0   |8641   |Anchorman: The Legend of Ron Burgundy (2004)          |
|332   |55820  |4.0   |55820  |No Country for Old Men (2007)                         |
|166   |48774  |4.5   |48774  |Children of Men (2006)                                |
|139   |5481   |1.0   |5481   |Austin Powers in Goldmember (2002)                    |
|116   |1036   |3.5   |1036   |Die Hard (1988)                                       |
|294   |2072   |1.0   |2072   |'burbs, The (1989)                                    |
|221   |2951   |4.5   |2951   |Fistful of Dollars, A (Per un pugno di dollari) (1964)|
|57    |95     |3.0   |95     |Broken Arrow (1996)                                   |
|125   |9010   |5.0   |9010   |Love Me If Y

In [ ]:
from pyspark.ml.feature import StringIndexer,IndexToString

In [ ]:
stringIndexer = StringIndexer(inputCol="title",outputCol="title_new")
model = stringIndexer.fit(df)
indexed_df = model.transform(df)

In [ ]:
indexed_df.orderBy(rand()).show()

+------+-------+------+-------+--------------------+---------+
|userId|movieId|rating|movieId|               title|title_new|
+------+-------+------+-------+--------------------+---------+
|   159|   6373|   4.0|   6373|Bruce Almighty (2...|    246.0|
|   180|   1213|   4.5|   1213|   Goodfellas (1990)|     76.0|
|   414|   1037|   3.0|   1037|Lawnmower Man, Th...|   1068.0|
|   186|   3527|   4.0|   3527|     Predator (1987)|    324.0|
|    19|   3108|   4.0|   3108|Fisher King, The ...|    712.0|
|   380|   3253|   3.0|   3253|Wayne's World (1992)|    327.0|
|   500|    597|   1.0|    597| Pretty Woman (1990)|     61.0|
|    84|    247|   4.0|    247|Heavenly Creature...|   1199.0|
|   480|   8984|   3.0|   8984|Ocean's Twelve (2...|    565.0|
|   292|  68954|   4.0|  68954|           Up (2009)|    119.0|
|   177|  64957|   3.5|  64957|Curious Case of B...|    691.0|
|    25|   5952|   5.0|   5952|Lord of the Rings...|     24.0|
|    50| 109487|   2.5| 109487| Interstellar (2014)|   

In [ ]:
indexed_df.groupBy('title_new').count().orderBy('count',ascending=False).show(10,False)

+---------+-----+
|title_new|count|
+---------+-----+
|0.0      |329  |
|1.0      |317  |
|2.0      |307  |
|3.0      |279  |
|4.0      |278  |
|5.0      |251  |
|6.0      |238  |
|7.0      |237  |
|8.0      |224  |
|9.0      |220  |
+---------+-----+
only showing top 10 rows



In [ ]:
train,test = indexed_df.randomSplit([0.80,0.20])
print(train.count(), test.count())

80937 19899


In [ ]:
from pyspark.ml.recommendation import ALS
rec = ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='title_new',ratingCol='rating',nonnegative=True,coldStartStrategy='drop')

rec_model = rec.fit(train)

In [ ]:
pred_ratings = rec_model.transform(test)

In [ ]:
pred_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- title_new: double (nullable = false)
 |-- prediction: float (nullable = false)



In [ ]:
pred_ratings.orderBy(rand()).show(10,False)

+------+-------+------+-------+---------------------------------------------+---------+----------+
|userId|movieId|rating|movieId|title                                        |title_new|prediction|
+------+-------+------+-------+---------------------------------------------+---------+----------+
|305   |109487 |3.5   |109487 |Interstellar (2014)                          |240.0    |4.952799  |
|237   |3617   |2.5   |3617   |Road Trip (2000)                             |610.0    |2.9385598 |
|63    |3481   |4.0   |3481   |High Fidelity (2000)                         |233.0    |3.2419999 |
|369   |27904  |5.0   |27904  |Scanner Darkly, A (2006)                     |1159.0   |3.3195517 |
|191   |269    |3.0   |269    |My Crazy Life (Mi vida loca) (1993)          |5771.0   |3.456217  |
|140   |44399  |1.0   |44399  |Shaggy Dog, The (2006)                       |5979.0   |1.2096078 |
|159   |4022   |3.0   |4022   |Cast Away (2000)                             |136.0    |3.3621314 |
|10    |69

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse",predictionCol='prediction',labelCol='rating')
rmse = evaluator.evaluate(pred_ratings)

In [ ]:
print(rmse)

1.0221893314517116


In [ ]:
unique_movies = indexed_df.select('title_new').distinct()
unique_movies.count()

9719

In [ ]:
unique_movies.show(15,False)

+---------+
|title_new|
+---------+
|299.0    |
|692.0    |
|2862.0   |
|305.0    |
|496.0    |
|596.0    |
|769.0    |
|934.0    |
|4066.0   |
|558.0    |
|1051.0   |
|3901.0   |
|3980.0   |
|7782.0   |
|6067.0   |
+---------+
only showing top 15 rows



In [ ]:
a = unique_movies.alias('a')

# perform recommendations for 85th user
user_id = 23
watched_movies = indexed_df.filter(indexed_df['userId'] == user_id).select('userId','title_new').distinct()

In [ ]:
watched_movies.count()

121

In [ ]:
b = watched_movies.alias('b')

In [ ]:
total_movies = a.join(b, a.title_new == b.title_new, how='left')

In [ ]:
total_movies.show(10,False)

+---------+------+---------+
|title_new|userId|title_new|
+---------+------+---------+
|299.0    |null  |null     |
|692.0    |null  |null     |
|2862.0   |null  |null     |
|305.0    |null  |null     |
|496.0    |null  |null     |
|596.0    |null  |null     |
|769.0    |null  |null     |
|934.0    |null  |null     |
|4066.0   |null  |null     |
|558.0    |null  |null     |
+---------+------+---------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import *
remain_movies = total_movies.where(col("b.title_new").isNull()).select(a.title_new,"UserId").distinct()

In [ ]:
remain_movies.count()
remain_movies.printSchema()

root
 |-- title_new: double (nullable = false)
 |-- UserId: integer (nullable = true)



In [ ]:
remain_movies = remain_movies.withColumn("userId",lit(user_id))
remain_movies.show(10,False)

+---------+------+
|title_new|userId|
+---------+------+
|299.0    |23    |
|692.0    |23    |
|2862.0   |23    |
|305.0    |23    |
|496.0    |23    |
|596.0    |23    |
|769.0    |23    |
|934.0    |23    |
|4066.0   |23    |
|558.0    |23    |
+---------+------+
only showing top 10 rows



In [ ]:
recommendations = rec_model.transform(remain_movies).orderBy('prediction',ascending=False)
recommendations.show(5,False)

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|2095.0   |23    |4.9942055 |
|2783.0   |23    |4.975934  |
|3218.0   |23    |4.808839  |
|4748.0   |23    |4.7894506 |
|2392.0   |23    |4.7411504 |
+---------+------+----------+
only showing top 5 rows



In [ ]:
movie_title = IndexToString(inputCol="title_new",outputCol='title',labels=model.labels)

In [ ]:
final_recs = movie_title.transform(recommendations)
final_recs.show(5,False)

+---------+------+----------+----------------------------------+
|title_new|userId|prediction|title                             |
+---------+------+----------+----------------------------------+
|2095.0   |23    |4.9942055 |Solaris (Solyaris) (1972)         |
|2783.0   |23    |4.975934  |Joe Versus the Volcano (1990)     |
|3218.0   |23    |4.808839  |Stranger Than Paradise (1984)     |
|4748.0   |23    |4.7894506 |Play Time (a.k.a. Playtime) (1967)|
|2392.0   |23    |4.7411504 |Public Enemies (2009)             |
+---------+------+----------+----------------------------------+
only showing top 5 rows

